In [1]:
import pandas as pd
import matplotlib.pyplot as plt

import numpy as np

import xgboost as xgb
import pickle

In [17]:
r16_tab1 = pd.read_excel('C:\\Users\\user\\deep_learning\\MDMS\\DataCompters\\R16\\R16_tab1_2022-11-15_19\\r16_tab1_16.xlsx')

In [ ]:
r16_tab1.loc[r16_tab1['type_mesure']=='max_puissance_reactive']

In [ ]:
r16_tab1.drop_duplicates(subset='type_mesure').loc[r16_tab1['date_heure_mesure']=='2022-11-15 09:29:25']

In [ ]:
set(r16_tab1.type_mesure)

In [3]:
r16_tab1=r16_tab1.loc[r16_tab1.type_mesure.isin(['max_courant', 'min_tension_n', 'max_tension_n', 
                                                 'max_tension_ph', 'min_puissance_reactive', 'max_puissance_reactive',
                                                 'max_puissance_active', 'max_puissance_apparente', 'min_courant', 'min_tension_ph',
                                                 'min_puissance_active'
                                                 ])]
                                               

In [4]:
mon_dictionnaire = {'max_courant': int(1), 'max_tension_n': int(2), 'max_tension_ph':int(3),  
                    'max_puissance_reactive':int(4), 'max_puissance_active':int(5), 'max_puissance_apparente':int(6), 
                    'min_courant':int(7), 'min_tension_n':int(8), 'min_tension_ph':int(9), 'min_puissance_reactive':int(10),
                    'min_puissance_active':int(11)
                    }


r16_tab1 = r16_tab1.set_index('date_heure_mesure')
r16_tab1.index = pd.to_datetime(r16_tab1.index)


def create_features(r16_tab1):
            """
            Create time series features based on time series index.
            """
            r16_tab1 = r16_tab1.copy()
            r16_tab1['heure_mesure'] = r16_tab1.index.hour
        #     r16_tab1['minute_mesure'] = (r16_tab1.index.minute)
        #     r16_tab1['seconde_mesure'] = r16_tab1.index.second
            r16_tab1['depart'] =r16_tab1.depart
            r16_tab1['type_mesure'] = r16_tab1.type_mesure
            r16_tab1['phase'] = r16_tab1.phase
            return r16_tab1

for type_mesure, encoder in mon_dictionnaire.items():
        # print("l'élément de clé", type_mesure, "vaut", encoder)
        r16_tab1_2 = r16_tab1.copy()

        r16_tab1_2['type_mesure'] = r16_tab1_2['type_mesure'].map({
                             type_mesure : encoder,
                            
                             },
                             na_action=None)

        r16_tab1_2['phase']= r16_tab1_2['phase'].replace([np.nan],[4])
        # set(r16_tab1.phase)
        r16_tab1_2 = r16_tab1_2.dropna(subset='type_mesure')
        r16_tab1_2['phase']=r16_tab1_2['phase'].astype(int)


        train = r16_tab1_2
        test = r16_tab1_2

        r16_tab1_2 = create_features(r16_tab1_2)
        train = create_features(train)
        test = create_features(test)

        FEATURES = ['depart', 'type_mesure', 'phase', 'heure_mesure']
        TARGET = 'mesure'

        X_train = train[FEATURES]
        y_train = train[TARGET]

        X_test = test[FEATURES]
        y_test = test[TARGET]

        reg = xgb.XGBRegressor(base_score=0.5, booster='gbtree',    
                            n_estimators=10000,
                            early_stopping_rounds=50,
                            objective='reg:linear',
                            max_depth=3,
                            learning_rate=0.001)
        reg.fit(X_train, y_train,
                eval_set=[(X_train, y_train), (X_test, y_test)],
                verbose=100)
                
        filename = f'r16_tab1_{type_mesure}'

        with open(f'{filename}.pkl', "wb") as f:
            pickle.dump(reg, f)

[14:16:58] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-03de431ba26204c4d-1/xgboost/xgboost-ci-windows/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:13.30849	validation_1-rmse:13.30849
[100]	validation_0-rmse:12.06830	validation_1-rmse:12.06830
[200]	validation_0-rmse:10.94412	validation_1-rmse:10.94412
[300]	validation_0-rmse:9.92515	validation_1-rmse:9.92515
[400]	validation_0-rmse:9.00160	validation_1-rmse:9.00160
[500]	validation_0-rmse:8.16457	validation_1-rmse:8.16457
[600]	validation_0-rmse:7.40603	validation_1-rmse:7.40603
[700]	validation_0-rmse:6.71869	validation_1-rmse:6.71869
[800]	validation_0-rmse:6.09592	validation_1-rmse:6.09592
[900]	validation_0-rmse:5.53176	validation_1-rmse:5.53176
[1000]	validation_0-rmse:5.02076	validation_1-rmse:5.02076
[1100]	validation_0-rmse:4.55802	validation_1-rmse:4.55802
[1200]	validation_0-rmse:4.13910	validation_1-rmse:4.13910
[1300]	v

In [ ]:
mon_dictionnaire = {'courant': int(1), 'courant_hta': int(2), 'desequilibre_courant':int(3),  
                    'tension_n':int(4), 'tension_ph':int(5), 'tension_ph_hta':int(6), 
                    'desequilibre_tension':int(7)}


In [ ]:
r16_tab1

In [5]:
test = r16_tab1.copy()

test['type_mesure'] = test['type_mesure'].map({
                             'courant' : int(1),                           
                             },
                             na_action=None)

test['phase']= test['phase'].replace(['N'],[4])
        # set(r16_tab1.phase)
test = test.dropna(subset='type_mesure')
test['phase']=test['phase'].astype(int)
test = create_features(test)
X_test = test[FEATURES]

with open('r16_Tfo1_courant.pkl', "rb") as f:
    reg = pickle.load(f)

test['prediction'] = reg.predict(X_test)
test.iloc[0:60]

FileNotFoundError: [Errno 2] No such file or directory: 'r16_Tfo1_courant.pkl'

In [ ]:
voir_test = test.loc[(test['type_mesure'] == 1) & (test['phase'] == 1)]

a = voir_test.index
b = voir_test.prediction
# c = voir_test.index
# d = voir_test.prediction

# plt.figure(figsize=(12, 8))
# plt.subplot(3,1,1)
plt.plot(a, b, c='yellow')
# plt.plot(c,d, c='yellow')
plt.show

In [ ]:
# r16_tab1 = r16_tab1.set_index('date_heure_mesure')
# r16_tab1.index = pd.to_datetime(r16_tab1.index)

def create_features(r16_tab1):
            """
            Create time series features based on time series index.
            """
            r16_tab1 = r16_tab1.copy()
            r16_tab1['heure_mesure'] = r16_tab1.index.hour
            r16_tab1['minute_mesure'] = (r16_tab1.index.minute)
        #     r16_tab1['seconde_mesure'] = r16_tab1.index.second
            r16_tab1['depart'] =r16_tab1.depart
            r16_tab1['type_mesure'] = r16_tab1.type_mesure
            r16_tab1['phase'] = r16_tab1.phase
            return r16_tab1

test = create_features(r16_tab1)

FEATURES = ['depart', 'type_mesure', 'phase', 'heure_mesure', 'minute_mesure']
TARGET = 'mesure'

# X_train = train[FEATURES]
# y_train = train[TARGET]

X_test = test[FEATURES]
y_test = test[TARGET]

with open('r16_tab1_tdh_tension.pkl', "rb") as f:
    reg = pickle.load(f)

test['prediction'] = reg.predict(X_test)
test

In [ ]:
mere = test.copy()
mere['laisse'] = 'seconde_mesure'
test

In [ ]:
mon_dictionnaire = {'courant': int(1), 'courant_hta': int(2), 'desequilibre_courant':int(3),  
                    'tension_n':int(4), 'tension_ph':int(5), 'tension_ph_hta':int(6), 
                    'desequilibre_tension':int(7)}


In [ ]:
r16_tab1.loc[r16_tab1['type_mesure']== 'tdh_courant'].drop_duplicates(subset='depart')

In [ ]:
r16_tab1.loc[(r16_tab1['type_mesure'] == 'tdh_courant')]

In [ ]:
'max_courant': int(1), 'max_tension_n': int(2), 'max_tension_ph':int(3),  
                    'max_puissance_reactive':int(4), 'max_puissance_active':int(5), 'max_puissance_apparente':int(6), 
                    'min_courant':int(7), 'min_tension_n':int(8), 'min_tension_ph':int(9), 'min_puissance_reactive':int(10),
                    'min_puissance_active':int(11)

In [ ]:
mon_dictionnaire = {'max_courant': int(1), 'max_tension_n': int(2), 'max_tension_ph':int(3),  
                    'max_puissance_reactive':int(4), 'max_puissance_active':int(5), 'max_puissance_apparente':int(6), 
                    'min_courant':int(7), 'min_tension_n':int(8), 'min_tension_ph':int(9), 'min_puissance_reactive':int(10),
                    'min_puissance_active':int(11)
                    }

In [1]:
import itertools
import numpy as np
import pandas as pd
import random

a = [[1,2,3,4], [1,2,3,4,5,6,7,8,9,10,11],[1,2,3,4]]
famille = list(itertools.product(*a))

my_data = []
for i in famille:
    my_data.append(list(i))

my_data = np.array(my_data)

column_names = ["depart","type_mesure", "phase"]  #4,10 -- 5,11,6
data_df = pd.DataFrame(my_data, columns=column_names)
data = data_df.drop((data_df.loc[(data_df['type_mesure'].isin([5,6,11])) & (data_df['phase'].isin([1,2,3]))].index) |
                    (data_df.loc[(data_df['type_mesure'].isin([1,2,3,7,8,9])) & (data_df['phase'].isin([4]))].index)
)
data


C:\Users\user\AppData\Local\Temp\ipykernel_9284\511675495.py:17: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead.
  data = data_df.drop((data_df.loc[(data_df['type_mesure'].isin([5,6,11])) & (data_df['phase'].isin([1,2,3]))].index) |


,depart,type_mesure,phase
0,1,1,1
1,1,1,2
2,1,1,3
4,1,2,1
5,1,2,2
...,...,...,...
168,4,10,1
169,4,10,2
170,4,10,3
171,4,10,4


In [2]:
import datetime
hr = int(datetime.datetime.now().strftime("%H"))
mn = int(datetime.datetime.now().strftime("%M"))
sd = int(datetime.datetime.now().strftime("%S"))
da_hr_ms = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
 
data['heure_mesure'] = hr
# data['minute_mesure'] = mn
# data['seconde_mesure'] = sd

In [22]:
r16_tab1.loc[r16_tab1['type_mesure']=='min_puissance_reactive']


,depart,type_mesure,phase,mesure,date_heure_mesure
166417,1,min_puissance_reactive,2,196.416,2022-11-16 03:29:25
166435,2,min_puissance_reactive,2,1219.476,2022-11-16 03:29:25
166447,3,min_puissance_reactive,2,243.754,2022-11-16 03:29:25
166462,4,min_puissance_reactive,2,1020.940,2022-11-16 03:29:25
166478,3,min_puissance_reactive,1,165.505,2022-11-16 03:29:25
...,...,...,...,...,...
1000310,3,min_puissance_reactive,2,482.209,2022-11-16 21:29:40
1000355,1,min_puissance_reactive,2,367.044,2022-11-16 21:29:40
1000372,3,min_puissance_reactive,1,191.117,2022-11-16 21:29:40
1000386,4,min_puissance_reactive,1,0.000,2022-11-16 21:29:40


In [3]:
import pickle

with open('r16_tab1_max_courant.pkl', "rb") as a:
    Mcrt = pickle.load(a)
with open('r16_tab1_max_tension_n.pkl', "rb") as b:
    Mten_n = pickle.load(b)
with open('r16_tab1_max_tension_ph.pkl', "rb") as c:
    Mten_ph = pickle.load(c)
with open('r16_tab1_max_puissance_reactive.pkl', "rb") as d:
    Mpui_rec = pickle.load(d)
with open('r16_tab1_max_puissance_active.pkl', "rb") as e:
    Mpui_act = pickle.load(e)
with open('r16_tab1_max_puissance_apparente.pkl', "rb") as f:
    Mpui_app = pickle.load(f)
with open('r16_tab1_min_courant.pkl', "rb") as g:
    mcrt = pickle.load(g)
with open('r16_tab1_min_tension_n.pkl', "rb") as h:
    mten_n = pickle.load(h)
with open('r16_tab1_min_tension_ph.pkl', "rb") as i:
    mten_ph = pickle.load(i)
with open('r16_tab1_min_puissance_reactive.pkl', "rb") as j:
    mpui_rec = pickle.load(j)
with open('r16_tab1_min_puissance_active.pkl', "rb") as k:
    mpui_act = pickle.load(k)

data_omo = data.copy()

data['type_mesure'].loc[data['type_mesure']==1] = Mcrt.predict(data_omo.loc[data['type_mesure']==1])
data['type_mesure'].loc[data['type_mesure']==2] = Mten_n.predict(data_omo.loc[data_omo['type_mesure']== 2])
data['type_mesure'].loc[data['type_mesure']==3] = Mten_ph.predict(data_omo.loc[data_omo['type_mesure']== 3])
data['type_mesure'].loc[data['type_mesure']==4] = Mpui_rec.predict(data_omo.loc[data_omo['type_mesure']== 4])
data['type_mesure'].loc[data['type_mesure']==5] = Mpui_act.predict(data_omo.loc[data_omo['type_mesure']== 5 ])
data['type_mesure'].loc[data['type_mesure']==6] = Mpui_app.predict(data_omo.loc[data_omo['type_mesure']== 6 ])
data['type_mesure'].loc[data['type_mesure']==7] = mcrt.predict(data_omo.loc[data_omo['type_mesure']== 7 ])
data['type_mesure'].loc[data['type_mesure']==8] = mten_n.predict(data_omo.loc[data_omo['type_mesure']== 8 ])
data['type_mesure'].loc[data['type_mesure']==9] = mten_ph.predict(data_omo.loc[data_omo['type_mesure']== 9 ])
data['type_mesure'].loc[data['type_mesure']==10] = mpui_rec.predict(data_omo.loc[data_omo['type_mesure']== 10 ])
data['type_mesure'].loc[data['type_mesure']==11] = mpui_act.predict(data_omo.loc[data_omo['type_mesure']== 11 ])



C:\Users\user\AppData\Local\Temp\ipykernel_9284\3438758216.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['type_mesure'].loc[data['type_mesure']==1] = Mcrt.predict(data_omo.loc[data['type_mesure']==1])


In [ ]:
mon_dictionnaire = {'max_courant': int(1), 'max_tension_n': int(2), 'max_tension_ph':int(3),  
                    'max_puissance_reactive':int(4), 'max_puissance_active':int(5), 'max_puissance_apparente':int(6), 
                    'min_courant':int(7), 'min_tension_n':int(8), 'min_tension_ph':int(9), 'min_puissance_reactive':int(10),
                    'min_puissance_active':int(11)
                    }
                    

In [4]:
data['mesure'] = data['type_mesure']
data['type_mesure'] = data_omo['type_mesure']


In [6]:
data.iloc[0:60]

,depart,type_mesure,phase,heure_mesure,mesure
0,1,1,1,14,14.558591
1,1,1,2,14,13.239385
2,1,1,3,14,13.628820
4,1,2,1,14,237.127975
5,1,2,2,14,234.828232
6,1,2,3,14,235.981018
8,1,3,1,14,408.863739
9,1,3,2,14,408.917786
10,1,3,3,14,410.139160
12,1,4,1,14,1819.788574


In [10]:
data['type_mesure'] = data['type_mesure'].map({
                            1:'max_courant', 2:'max_tension_n', 3:'max_tension_ph', 
                            4:'max_puissance_reactive', 5:'max_puissance_active', 6:'max_puissance_apparente',
                            7:'min_courant', 8:'min_tension_n', 9:'min_tension_ph', 10:'min_puissance_reactive',
                            11:'min_puissance_active'                  
                             },
                             na_action=None)

data['phase']= data['phase'].replace([4],[''])


In [11]:
data.iloc[100:130]

,depart,type_mesure,phase,mesure,date_heure_mesure
151,4,max_puissance_active,,9254.974609,2022-12-05 14:46:13
155,4,max_puissance_apparente,,0.000041,2022-12-05 14:46:13
156,4,min_courant,1,2.269870,2022-12-05 14:46:13
157,4,min_courant,2,8.599213,2022-12-05 14:46:13
158,4,min_courant,3,7.729572,2022-12-05 14:46:13
160,4,min_tension_n,1,233.478195,2022-12-05 14:46:13
161,4,min_tension_n,2,231.148453,2022-12-05 14:46:13
162,4,min_tension_n,3,232.491226,2022-12-05 14:46:13
164,4,min_tension_ph,1,401.283783,2022-12-05 14:46:13
165,4,min_tension_ph,2,402.453888,2022-12-05 14:46:13


In [8]:
data['date_heure_mesure'] = da_hr_ms
data = data.drop(columns=['heure_mesure'])


In [9]:
data

,depart,type_mesure,phase,mesure,date_heure_mesure
0,1,1,1,14.558591,2022-12-05 14:46:13
1,1,1,2,13.239385,2022-12-05 14:46:13
2,1,1,3,13.628820,2022-12-05 14:46:13
4,1,2,1,237.127975,2022-12-05 14:46:13
5,1,2,2,234.828232,2022-12-05 14:46:13
...,...,...,...,...,...
168,4,10,1,-29.167780,2022-12-05 14:46:13
169,4,10,2,707.134399,2022-12-05 14:46:13
170,4,10,3,682.964661,2022-12-05 14:46:13
171,4,10,4,1486.082031,2022-12-05 14:46:13


In [12]:

data['id_poste'] = 'P0016.210.043'
data['zone_poste'] = 'tab1'
data['classification'] = 'CLA_BT'
# data['classification'].loc[data['type_mesure'].isin(['tension_ph_hta', 'courant_hta'])] = 'CLA_HTA'
data['date_heure_transfert'] = da_hr_ms
data['unite_mesure'] = data['type_mesure']
data['unite_mesure'].loc[data['unite_mesure'].isin(['max_courant', 'min_courant', '', ''])] = 'MWh'
data['unite_mesure'].loc[data['unite_mesure'].isin(['max_tension_n', 'min_tension_n', 'max_tension_ph', 'min_tension_ph'])] = 'Hz'
data['unite_mesure'].loc[data['unite_mesure'].isin(['min_puissance_active', 'max_puissance_active', '', '',''])] = 'W'
data['unite_mesure'].loc[data['unite_mesure'].isin(['max_puissance_apparente'])] = 'VA'
data['unite_mesure'].loc[data['unite_mesure'].isin(['max_puissance_reactive', 'min_puissance_reactive'])] = 'VAR'
# data['unite_mesure'].loc[data['unite_mesure'].isin(['facteur_puissance', 'taux_de_charge', 'tdh_courant', 'tdh_tension'])] = '%'
# data['phase'] = data['phase'].astype(int)
data['phase'] = data['phase'].astype(str)
data['description_mesure'] = 'mesure - ' + data['type_mesure'] + ' phase ' +  data['phase']


C:\Users\user\AppData\Local\Temp\ipykernel_9284\3580076241.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['unite_mesure'].loc[data['unite_mesure'].isin(['max_courant', 'min_courant', '', ''])] = 'MWh'
C:\Users\user\AppData\Local\Temp\ipykernel_9284\3580076241.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['unite_mesure'].loc[data['unite_mesure'].isin(['max_tension_n', 'min_tension_n', 'max_tension_ph', 'min_tension_ph'])] = 'Hz'
C:\Users\user\AppData\Local\Temp\ipykernel_9284\3580076241.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats 

In [13]:
data

,depart,type_mesure,phase,mesure,date_heure_mesure,id_poste,zone_poste,classification,date_heure_transfert,unite_mesure,description_mesure
0,1,max_courant,1,14.558591,2022-12-05 14:46:13,P0016.210.043,tab1,CLA_BT,2022-12-05 14:46:13,MWh,mesure - max_courant phase 1
1,1,max_courant,2,13.239385,2022-12-05 14:46:13,P0016.210.043,tab1,CLA_BT,2022-12-05 14:46:13,MWh,mesure - max_courant phase 2
2,1,max_courant,3,13.628820,2022-12-05 14:46:13,P0016.210.043,tab1,CLA_BT,2022-12-05 14:46:13,MWh,mesure - max_courant phase 3
4,1,max_tension_n,1,237.127975,2022-12-05 14:46:13,P0016.210.043,tab1,CLA_BT,2022-12-05 14:46:13,Hz,mesure - max_tension_n phase 1
5,1,max_tension_n,2,234.828232,2022-12-05 14:46:13,P0016.210.043,tab1,CLA_BT,2022-12-05 14:46:13,Hz,mesure - max_tension_n phase 2
...,...,...,...,...,...,...,...,...,...,...,...
168,4,min_puissance_reactive,1,-29.167780,2022-12-05 14:46:13,P0016.210.043,tab1,CLA_BT,2022-12-05 14:46:13,VAR,mesure - min_puissance_reactive phase 1
169,4,min_puissance_reactive,2,707.134399,2022-12-05 14:46:13,P0016.210.043,tab1,CLA_BT,2022-12-05 14:46:13,VAR,mesure - min_puissance_reactive phase 2
170,4,min_puissance_reactive,3,682.964661,2022-12-05 14:46:13,P0016.210.043,tab1,CLA_BT,2022-12-05 14:46:13,VAR,mesure - min_puissance_reactive phase 3
171,4,min_puissance_reactive,,1486.082031,2022-12-05 14:46:13,P0016.210.043,tab1,CLA_BT,2022-12-05 14:46:13,VAR,mesure - min_puissance_reactive phase


In [14]:
data = data[[
    'id_poste', 'zone_poste', 'classification', 
    'type_mesure', 'description_mesure', 'phase', 'mesure',
    'unite_mesure', 'date_heure_mesure','date_heure_transfert'
    ]]
    
data_log = data.to_json(orient = 'records', lines=True)


In [15]:
files_name = 'R16_Tab1_{}.log'.format(datetime.datetime.now().strftime("%Y-%m-%d"))

file = open(files_name, "a+")
file.write(data_log)
file.close()


In [ ]:
test['prediction'] = reg.predict(X_test)


In [ ]:
test.loc[(test['type_mesure']==1) & (test['depart']==2)].iloc[0:60]
